In [47]:
import sys
sys.path.append('../')
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize 

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### Loading Data 

Import json files and convert to dataframe

** better to use json.load or pandas.read_json?

In [26]:
path2data = '../cleaning/data-json/matches/'
input_file = open(path2data + 'league_id_10749_ti9_matchdata.json') 
data = json.load(input_file)
input_file.close()

In [29]:
big_df = json_normalize(data)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



### Create Dataframe for match results

This will cover general id columns and basic match results such as match id, league id, dire/radiant teams, and if radiant won

Method 1: since data is saved as json, use pd.read_json

In [22]:
path2data = '../cleaning/data-json/matches/'
data2 = pd.read_json(path2data + 'league_id_10749_ti9_matchdata.json')
data2.head(1)

,match_id,barracks_status_dire,barracks_status_radiant,chat,cluster,cosmetics,dire_score,dire_team_id,draft_timings,duration,engine,first_blood_time,game_mode,human_players,leagueid,lobby_type,match_seq_num,negative_votes,objectives,picks_bans,positive_votes,radiant_gold_adv,radiant_score,radiant_team_id,radiant_win,radiant_xp_adv,skill,start_time,teamfights,tower_status_dire,tower_status_radiant,version,replay_salt,series_id,series_type,league,radiant_team,dire_team,players,patch,all_word_counts,my_word_counts,throw,loss,replay_url,region,comeback,stomp
0,4968150000,0,63,"[{'time': -960, 'type': 'chatwheel', 'key': '1...",308,"{'647': 130, '4670': 130, '4794': 4, '5181': 1...",42,36.0,"[{'order': 1, 'pick': False, 'active_team': 2,...",4934,1,29,2,10,10749,1,4167421134,369,"[{'time': 29, 'type': 'CHAT_MESSAGE_FIRSTBLOOD...","[{'is_pick': False, 'hero_id': 106, 'team': 1,...",4918,"[0, 495, 762, 808, 878, 1145, 2106, 2059, 2167...",54,39.0,True,"[0, 394, 247, 561, 224, 839, 1582, 1728, 1771,...",NaN,2019-08-15 11:20:48,"[{'start': 915, 'end': 959, 'last_death': 944,...",0,1796,21,37509985,358264,1,"{'leagueid': 10749, 'ticket': None, 'banner': ...","{'team_id': 39, 'name': 'Evil Geniuses', 'tag'...","{'team_id': 36, 'name': 'Natus Vincere', 'tag'...","[{'match_id': 4968150000, 'player_slot': 0, 'a...",41,"{'gl': 1, 'hf': 1, 'gg': 5, 'wp': 1}",{},9155.0,18713.0,http://replay308.valve.net/570/4968150000_3750...,NaN,NaN,NaN


In [34]:
match_df = big_df[['match_id','duration', 'leagueid','match_seq_num','series_id', 'series_type', 'patch', 'dire_team_id', 'radiant_team_id', 'radiant_win' ]]
match_df.info()
#match_df.isnull

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   match_id         461 non-null    int64  
 1   duration         461 non-null    int64  
 2   leagueid         461 non-null    int64  
 3   match_seq_num    461 non-null    int64  
 4   series_id        461 non-null    int64  
 5   series_type      461 non-null    int64  
 6   patch            461 non-null    int64  
 7   dire_team_id     449 non-null    float64
 8   radiant_team_id  451 non-null    float64
 9   radiant_win      461 non-null    bool   
dtypes: bool(1), float64(2), int64(7)
memory usage: 33.0 KB


In [45]:
match_df.shape

(461, 10)

In [46]:
match_df.index('match_id')

TypeError: 'RangeIndex' object is not callable

In [126]:
#match_df.head()

In [104]:
#match_df.sort_values(by='match_id', ascending=True)

Method 2: load json file and normalize using pandas.json_normalize.

** Dunno if there is any significant advantage to doing it one way or the other. 

In [32]:
#json_struct = json.loads(data2.to_json(orient="records"))    
#df_flat = pd.io.json.json_normalize(json_struct) #use pd.io.json

In [38]:
picks_df = pd.json_normalize(data, 'picks_bans', max_level=0) # Normalize and extract the 'picks_bans' object from the json file.
#picks_df.head(100)
#picks_df.shape

In [39]:
df = picks_df[['match_id', 'order', 'is_pick', 'team', 'hero_id']]
#df_top = df.head(20) 


In [111]:
df_asc = df.sort_values(by='match_id', ascending=True)
df_asc.head(22)

,match_id,order,is_pick,team,hero_id
9095,4888646940,10,False,1,10
9105,4888646940,20,True,1,44
9104,4888646940,19,False,1,46
9103,4888646940,18,False,0,8
9102,4888646940,17,True,1,52
9101,4888646940,16,True,0,38
9100,4888646940,15,True,1,88
9099,4888646940,14,True,0,86
9098,4888646940,13,False,0,76
9097,4888646940,12,False,1,69


In [40]:
df = df.astype(str)
df = df.groupby('match_id')['is_pick','team','hero_id'].agg(' '.join)
#df.head(100)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [42]:
#new = df['order'].str.split(' ', expand=True)
new1 = df['is_pick'].str.split(' ', expand=True)
new2 = df['team'].str.split(' ', expand=True)
new3 = df['hero_id'].str.split(' ', expand=True)
#new1, new2, new3

In [87]:
#nameless_df= new1.join(new2).join(new3)

In [43]:
# Get column labels for the columns 
total_picks = 21 # recall range is 0-21 = 22 picks
new1.columns = ['1is_pick', '2is_pick', '3is_pick', '4is_pick', '5is_pick', '6is_pick', '7is_pick','8is_pick','9is_pick','10is_pick','11is_pick','12is_pick','13is_pick','14is_pick','15is_pick','16is_pick','17is_pick','18is_pick','19is_pick','20is_pick','21is_pick', '22is_pick']
new2.columns = ['1team', '2team', '3team', '4team', '5team', '6team', '7team', '8team', '9team', '10team', '11team', '12team', '13team', '14team', '15team', '16team', '17team', '18team', '19team', '20team', '21team', '22team']
new3.columns = ['1hero_id', '2hero_id', '3hero_id', '4hero_id', '5hero_id', '6hero_id', '7hero_id', '8hero_id', '9hero_id', '10hero_id', '11hero_id', '12hero_id', '13hero_id', '14hero_id', '15hero_id', '16hero_id', '17hero_id', '18hero_id', '19hero_id', '20hero_id', '21hero_id',
               '22hero_id']

In [44]:
picks2 = new1.join(new2).join(new3)
#picks_df = new1.join(new2).join(new3)
#picks_df.dtypes
#picks_df.head(100)
picks2.shape

(460, 66)

Quick eyeball inspection to check if the draft information is preserved:


Comparing match_id = 4888646940, we can see from above that the first pick (order=0) information was: 
- first pick (1ban): '4888646940	0	False	1	66'; w/c lines up with 1is_pick, 1hero_id, etc...
- last pick: 9106	4888646940	21	True	0	67; w/c lines up with 22is_pick, 22hero_id, etc...

So, at least by a quick visual inspection, seems like the drafting data has been successfully cleaned. 

In [98]:
#data.sort_index(ascending=True)
data_top = data.head(10)  
    
# display  
data_top  

,1is_pick,2is_pick,3is_pick,4is_pick,5is_pick,6is_pick,7is_pick,8is_pick,9is_pick,10is_pick,11is_pick,12is_pick,13is_pick,14is_pick,15is_pick,16is_pick,17is_pick,18is_pick,19is_pick,20is_pick,21is_pick,22is_pick,1team,2team,3team,4team,5team,6team,7team,8team,9team,10team,11team,12team,13team,14team,15team,16team,17team,18team,19team,20team,21team,22team,1hero_id,2hero_id,3hero_id,4hero_id,5hero_id,6hero_id,7hero_id,8hero_id,9hero_id,10hero_id,11hero_id,12hero_id,13hero_id,14hero_id,15hero_id,16hero_id,17hero_id,18hero_id,19hero_id,20hero_id,21hero_id,22hero_id
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4888646940,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,66,71,91,53,33,57,121,37,106,96,10,54,69,76,86,88,38,52,8,46,44,67
4888686136,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,76,88,121,106,86,33,68,66,42,89,52,7,99,65,16,71,57,98,25,50,114,39
4888732113,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,66,121,91,112,57,37,33,96,86,75,8,65,45,15,46,106,111,29,93,70,95,114
4888772755,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,18,91,43,97,78,37,33,86,112,64,17,89,39,10,54,38,29,15,52,76,8,45
4888859291,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,1,0,1,0,0,1,18,88,65,71,66,33,91,79,96,72,10,13,54,52,95,121,97,69,25,15,23,76
4888936676,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,57,91,71,37,18,106,97,33,27,41,43,64,8,121,100,112,42,65,23,45,52,94
4888974952,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,91,43,66,18,33,37,27,106,121,78,10,54,55,76,7,102,36,17,6,73,8,95
4889038294,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,1,0,1,0,0,1,76,91,121,66,86,33,68,106,111,10,89,57,96,7,27,71,38,98,23,70,42,11
4889115579,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,66,91,18,33,121,37,106,57,27,97,76,15,42,41,112,8,10,79,60,36,96,52


In [113]:
data.shape

(460, 66)

### Replacing T/F values

Convert T/F to boolean 

In [115]:
# source: https://github.com/codexponent/dota2-draft-analysis/blob/master/draft-ml.ipynb
# # # Converting boolean value into number
boolean_values = 22
for i in range(boolean_values):
    pick_id = '{}is_pick'.format(i+1)
    data[pick_id] = data[pick_id].apply(lambda x: 1 if x else 0)

In [116]:
data.head(10)

,1is_pick,2is_pick,3is_pick,4is_pick,5is_pick,6is_pick,7is_pick,8is_pick,9is_pick,10is_pick,11is_pick,12is_pick,13is_pick,14is_pick,15is_pick,16is_pick,17is_pick,18is_pick,19is_pick,20is_pick,21is_pick,22is_pick,1team,2team,3team,4team,5team,6team,7team,8team,9team,10team,11team,12team,13team,14team,15team,16team,17team,18team,19team,20team,21team,22team,1hero_id,2hero_id,3hero_id,4hero_id,5hero_id,6hero_id,7hero_id,8hero_id,9hero_id,10hero_id,11hero_id,12hero_id,13hero_id,14hero_id,15hero_id,16hero_id,17hero_id,18hero_id,19hero_id,20hero_id,21hero_id,22hero_id
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4888646940,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,66,71,91,53,33,57,121,37,106,96,10,54,69,76,86,88,38,52,8,46,44,67
4888686136,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,76,88,121,106,86,33,68,66,42,89,52,7,99,65,16,71,57,98,25,50,114,39
4888732113,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,66,121,91,112,57,37,33,96,86,75,8,65,45,15,46,106,111,29,93,70,95,114
4888772755,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,18,91,43,97,78,37,33,86,112,64,17,89,39,10,54,38,29,15,52,76,8,45
4888859291,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,1,0,1,0,0,1,18,88,65,71,66,33,91,79,96,72,10,13,54,52,95,121,97,69,25,15,23,76
4888936676,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,57,91,71,37,18,106,97,33,27,41,43,64,8,121,100,112,42,65,23,45,52,94
4888974952,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,91,43,66,18,33,37,27,106,121,78,10,54,55,76,7,102,36,17,6,73,8,95
4889038294,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,1,0,1,0,0,1,76,91,121,66,86,33,68,106,111,10,89,57,96,7,27,71,38,98,23,70,42,11
4889115579,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0,66,91,18,33,121,37,106,57,27,97,76,15,42,41,112,8,10,79,60,36,96,52


In [117]:
data.shape

(460, 66)

In [30]:
data.columns.tolist()

['1is_pick',
 '2is_pick',
 '3is_pick',
 '4is_pick',
 '5is_pick',
 '6is_pick',
 '7is_pick',
 '8is_pick',
 '9is_pick',
 '10is_pick',
 '11is_pick',
 '12is_pick',
 '13is_pick',
 '14is_pick',
 '15is_pick',
 '16is_pick',
 '17is_pick',
 '18is_pick',
 '19is_pick',
 '20is_pick',
 '21is_pick',
 '22is_pick',
 '1team',
 '2team',
 '3team',
 '4team',
 '5team',
 '6team',
 '7team',
 '8team',
 '9team',
 '10team',
 '11team',
 '12team',
 '13team',
 '14team',
 '15team',
 '16team',
 '17team',
 '18team',
 '19team',
 '20team',
 '21team',
 '22team',
 '1hero_id',
 '2hero_id',
 '3hero_id',
 '4hero_id',
 '5hero_id',
 '6hero_id',
 '7hero_id',
 '8hero_id',
 '9hero_id',
 '10hero_id',
 '11hero_id',
 '12hero_id',
 '13hero_id',
 '14hero_id',
 '15hero_id',
 '16hero_id',
 '17hero_id',
 '18hero_id',
 '19hero_id',
 '20hero_id',
 '21hero_id',
 '22hero_id']

Re-organize columns according to prefix order:
- pretty sure there's a better way to do this 

In [122]:
picks_df = data[['1is_pick', '1team', '1hero_id',
               '2is_pick', '2team', '2hero_id', 
               '3is_pick', '3team', '3hero_id', 
               '4is_pick', '4team', '4hero_id', 
               '5is_pick', '5team', '5hero_id', 
               '6is_pick', '6team', '6hero_id', 
               '7is_pick', '7team', '7hero_id', 
               '8is_pick', '8team', '8hero_id', 
               '9is_pick', '9team', '9hero_id', 
               '10is_pick', '10team', '10hero_id', 
               '11is_pick', '11team', '11hero_id', 
               '12is_pick', '12team', '12hero_id',
               '13is_pick', '13team', '13hero_id', 
               '14is_pick', '14team', '14hero_id', 
               '15is_pick', '15team', '15hero_id', 
               '16is_pick', '16team', '16hero_id', 
               '17is_pick', '17team', '17hero_id',
               '18is_pick', '18team', '18hero_id', 
               '19is_pick', '19team', '19hero_id',
               '20is_pick', '20team', '20hero_id',
               '21is_pick', '21team', '21hero_id', 
                '22is_pick', '22team', '22hero_id']]

In [123]:
picks_df.shape

(460, 66)

In [124]:
picks_df.head(30)

,1is_pick,1team,1hero_id,2is_pick,2team,2hero_id,3is_pick,3team,3hero_id,4is_pick,4team,4hero_id,5is_pick,5team,5hero_id,6is_pick,6team,6hero_id,7is_pick,7team,7hero_id,8is_pick,8team,8hero_id,9is_pick,9team,9hero_id,10is_pick,10team,10hero_id,11is_pick,11team,11hero_id,12is_pick,12team,12hero_id,13is_pick,13team,13hero_id,14is_pick,14team,14hero_id,15is_pick,15team,15hero_id,16is_pick,16team,16hero_id,17is_pick,17team,17hero_id,18is_pick,18team,18hero_id,19is_pick,19team,19hero_id,20is_pick,20team,20hero_id,21is_pick,21team,21hero_id,22is_pick,22team,22hero_id
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4888646940,1,1,66,1,0,71,1,1,91,1,0,53,1,1,33,1,0,57,1,1,121,1,0,37,1,0,106,1,1,96,1,1,10,1,0,54,1,1,69,1,0,76,1,0,86,1,1,88,1,0,38,1,1,52,1,0,8,1,1,46,1,1,44,1,0,67
4888686136,1,1,76,1,0,88,1,1,121,1,0,106,1,1,86,1,0,33,1,1,68,1,0,66,1,0,42,1,1,89,1,1,52,1,0,7,1,1,99,1,0,65,1,0,16,1,1,71,1,0,57,1,1,98,1,0,25,1,1,50,1,1,114,1,0,39
4888732113,1,1,66,1,0,121,1,1,91,1,0,112,1,1,57,1,0,37,1,1,33,1,0,96,1,0,86,1,1,75,1,1,8,1,0,65,1,1,45,1,0,15,1,0,46,1,1,106,1,0,111,1,1,29,1,0,93,1,1,70,1,1,95,1,0,114
4888772755,1,1,18,1,0,91,1,1,43,1,0,97,1,1,78,1,0,37,1,1,33,1,0,86,1,0,112,1,1,64,1,1,17,1,0,89,1,1,39,1,0,10,1,0,54,1,1,38,1,0,29,1,1,15,1,0,52,1,1,76,1,1,8,1,0,45
4888859291,1,0,18,1,1,88,1,0,65,1,1,71,1,0,66,1,1,33,1,0,91,1,1,79,1,1,96,1,0,72,1,0,10,1,1,13,1,0,54,1,1,52,1,1,95,1,0,121,1,1,97,1,0,69,1,1,25,1,0,15,1,0,23,1,1,76
4888936676,1,1,57,1,0,91,1,1,71,1,0,37,1,1,18,1,0,106,1,1,97,1,0,33,1,0,27,1,1,41,1,1,43,1,0,64,1,1,8,1,0,121,1,0,100,1,1,112,1,0,42,1,1,65,1,0,23,1,1,45,1,1,52,1,0,94
4888974952,1,1,91,1,0,43,1,1,66,1,0,18,1,1,33,1,0,37,1,1,27,1,0,106,1,0,121,1,1,78,1,1,10,1,0,54,1,1,55,1,0,76,1,0,7,1,1,102,1,0,36,1,1,17,1,0,6,1,1,73,1,1,8,1,0,95
4889038294,1,0,76,1,1,91,1,0,121,1,1,66,1,0,86,1,1,33,1,0,68,1,1,106,1,1,111,1,0,10,1,0,89,1,1,57,1,0,96,1,1,7,1,1,27,1,0,71,1,1,38,1,0,98,1,1,23,1,0,70,1,0,42,1,1,11
4889115579,1,1,66,1,0,91,1,1,18,1,0,33,1,1,121,1,0,37,1,1,106,1,0,57,1,0,27,1,1,97,1,1,76,1,0,15,1,1,42,1,0,41,1,0,112,1,1,8,1,0,10,1,1,79,1,0,60,1,1,36,1,1,96,1,0,52


In [83]:
# From a previous attempt to re-order
#picks_df.shape 

(10119, 6)

In [127]:
ti9_picks = picks_df
ti9_matchhistory = match_df

In [129]:
ti9_picks.to_csv('data/ti9_picks.csv')
ti9_matchhistory.to_csv('data/ti9_matchhistory.csv') 